# Welcome to the start of your adventure in Agentic AI

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Are you ready for action??</h2>
            <span style="color:#ff7800;">Have you completed all the setup steps in the <a href="../setup/">setup</a> folder?<br/>
            Have you read the <a href="../README.md">README</a>? Many common questions are answered here!<br/>
            Have you checked out the guides in the <a href="../guides/01_intro.ipynb">guides</a> folder?<br/>
            Well in that case, you're ready!!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">This code is a live resource - keep an eye out for my updates</h2>
            <span style="color:#00bfff;">I push updates regularly. As people ask questions or have problems, I add more examples and improve explanations. As a result, the code below might not be identical to the videos, as I've added more steps and better comments. Consider this like an interactive book that accompanies the lectures.<br/><br/>
            I try to send emails regularly with important updates related to the course. You can find this in the 'Announcements' section of Udemy in the left sidebar. You can also choose to receive my emails via your Notification Settings in Udemy. I'm respectful of your inbox and always try to add value with my emails!
            </span>
        </td>
    </tr>
</table>

### And please do remember to contact me if I can help

And I love to connect: https://www.linkedin.com/in/eddonner/


### New to Notebooks like this one? Head over to the guides folder!

Just to check you've already added the Python and Jupyter extensions to Cursor, if not already installed:
- Open extensions (View >> extensions)
- Search for python, and when the results show, click on the ms-python one, and Install it if not already installed
- Search for jupyter, and when the results show, click on the Microsoft one, and Install it if not already installed  
Then View >> Explorer to bring back the File Explorer.

And then:
1. Click where it says "Select Kernel" near the top right, and select the option called `.venv (Python 3.12.9)` or similar, which should be the first choice or the most prominent choice. You may need to choose "Python Environments" first.
2. Click in each "cell" below, starting with the cell immediately below this text, and press Shift+Enter to run
3. Enjoy!

After you click "Select Kernel", if there is no option like `.venv (Python 3.12.9)` then please do the following:  
1. On Mac: From the Cursor menu, choose Settings >> VS Code Settings (NOTE: be sure to select `VSCode Settings` not `Cursor Settings`);  
On Windows PC: From the File menu, choose Preferences >> VS Code Settings(NOTE: be sure to select `VSCode Settings` not `Cursor Settings`)  
2. In the Settings search bar, type "venv"  
3. In the field "Path to folder with a list of Virtual Environments" put the path to the project root, like C:\Users\username\projects\agents (on a Windows PC) or /Users/username/projects/agents (on Mac or Linux).  
And then try again.

Having problems with missing Python versions in that list? Have you ever used Anaconda before? It might be interferring. Quit Cursor, bring up a new command line, and make sure that your Anaconda environment is deactivated:    
`conda deactivate`  
And if you still have any problems with conda and python versions, it's possible that you will need to run this too:  
`conda config --set auto_activate_base false`  
and then from within the Agents directory, you should be able to run `uv python list` and see the Python 3.12 version.

In [1]:
# First let's do an import. If you get an Import Error, double check that your Kernel is correct..

from dotenv import load_dotenv


In [2]:
# Next it's time to load the API keys into environment variables
# If this returns false, see the next cell!

load_dotenv(override=True)

True

### Wait, did that just output `False`??

If so, the most common reason is that you didn't save your `.env` file after adding the key! Be sure to have saved.

Also, make sure the `.env` file is named precisely `.env` and is in the project root directory (`agents`)

By the way, your `.env` file should have a stop symbol next to it in Cursor on the left, and that's actually a good thing: that's Cursor saying to you, "hey, I realize this is a file filled with secret information, and I'm not going to send it to an external AI to suggest changes, because your keys should not be shown to anyone else."

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Final reminders</h2>
            <span style="color:#ff7800;">1. If you're not confident about Environment Variables or Web Endpoints / APIs, please read Topics 3 and 5 in this <a href="../guides/04_technical_foundations.ipynb">technical foundations guide</a>.<br/>
            2. If you want to use AIs other than OpenAI, like Gemini, DeepSeek or Ollama (free), please see the first section in this <a href="../guides/09_ai_apis_and_ollama.ipynb">AI APIs guide</a>.<br/>
            3. If you ever get a Name Error in Python, you can always fix it immediately; see the last section of this <a href="../guides/06_python_foundations.ipynb">Python Foundations guide</a> and follow both tutorials and exercises.<br/>
            </span>
        </td>
    </tr>
</table>

In [3]:
# Check the key - if you're not using OpenAI, check whichever key you're using! Ollama doesn't need a key.

import os
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")
    


OpenAI API Key exists and begins sk-proj-


In [4]:
# And now - the all important import statement
# If you get an import error - head over to troubleshooting in the Setup folder
# Even for other LLM providers like Gemini, you still use this OpenAI import - see Guide 9 for why

from openai import OpenAI

In [5]:
# And now we'll create an instance of the OpenAI class
# If you're not sure what it means to create an instance of a class - head over to the guides folder (guide 6)!
# If you get a NameError - head over to the guides folder (guide 6)to learn about NameErrors - always instantly fixable
# If you're not using OpenAI, you just need to slightly modify this - precise instructions are in the AI APIs guide (guide 9)

openai = OpenAI()

In [6]:
# Create a list of messages in the familiar OpenAI format

messages = [{"role": "user", "content": "What is 2+2?"}]

In [7]:
# And now call it! Any problems, head to the troubleshooting guide
# This uses GPT 4.1 nano, the incredibly cheap model
# The APIs guide (guide 9) has exact instructions for using even cheaper or free alternatives to OpenAI
# If you get a NameError, head to the guides folder (guide 6) to learn about NameErrors - always instantly fixable

response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

print(response.choices[0].message.content)


2 + 2 equals 4.


In [8]:
# And now - let's ask for a question:

question = "Please propose a hard, challenging question to assess someone's IQ. Respond only with the question."
messages = [{"role": "user", "content": question}]


In [9]:
# ask it - this uses GPT 4.1 mini, still cheap but more powerful than nano

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

question = response.choices[0].message.content

print(question)


If three people can complete a task in 4 hours, and five people can complete the same task in 2.5 hours, how long would it take seven people working at the same rate to complete the task? Explain your reasoning.


In [10]:
# form a new messages list
messages = [{"role": "user", "content": question}]


In [11]:
# Ask it again

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=messages
)

answer = response.choices[0].message.content
print(answer)


Let's analyze the problem step-by-step.

---

### Given:
- 3 people can complete the task in 4 hours.
- 5 people can complete the same task in 2.5 hours.
- We want to find how long it would take 7 people to complete the task.

---

### Assumptions:
- All people work at the same rate.
- The work is divisible and can be completed proportionally by the number of workers.

---

### Step 1: Define the total amount of work
Let's denote the total work as \( W \). The unit of \( W \) can be "task units," where completing the task means completing \( W = 1 \) unit.

---

### Step 2: Define the rate of each person per hour
Let \( r \) be the rate at which **each person** works, in tasks per hour.

- Total rate for 3 people = \( 3r \)
- Total rate for 5 people = \( 5r \)

---

### Step 3: Use the given data to find \( r \) or check consistency

Using the first data point:
\[
\text{Time} = \frac{\text{Work}}{\text{Rate}} \implies 4 = \frac{1}{3r} \implies 3r = \frac{1}{4} \implies r = \frac{1}{12}

In [12]:
from IPython.display import Markdown, display

display(Markdown(answer))



Let's analyze the problem step-by-step.

---

### Given:
- 3 people can complete the task in 4 hours.
- 5 people can complete the same task in 2.5 hours.
- We want to find how long it would take 7 people to complete the task.

---

### Assumptions:
- All people work at the same rate.
- The work is divisible and can be completed proportionally by the number of workers.

---

### Step 1: Define the total amount of work
Let's denote the total work as \( W \). The unit of \( W \) can be "task units," where completing the task means completing \( W = 1 \) unit.

---

### Step 2: Define the rate of each person per hour
Let \( r \) be the rate at which **each person** works, in tasks per hour.

- Total rate for 3 people = \( 3r \)
- Total rate for 5 people = \( 5r \)

---

### Step 3: Use the given data to find \( r \) or check consistency

Using the first data point:
\[
\text{Time} = \frac{\text{Work}}{\text{Rate}} \implies 4 = \frac{1}{3r} \implies 3r = \frac{1}{4} \implies r = \frac{1}{12}
\]

Using the second data point:
\[
2.5 = \frac{1}{5r} \implies 5r = \frac{1}{2.5} = 0.4 \implies r = \frac{0.4}{5} = 0.08
\]

---

### Step 4: Check if the two values of \( r \) are the same
- From the first data: \( r = \frac{1}{12} \approx 0.0833 \)
- From the second data: \( r = 0.08 \)

The two values are close but not exactly the same, which suggests the rate might vary slightly or there might be rounding errors. For simplicity, and because the problem usually expects consistent rates, let's average the two rates:

\[
r \approx \frac{0.0833 + 0.08}{2} = 0.08165
\]

---

### Step 5: Find the total time for 7 people
Total rate for 7 people:
\[
7r = 7 \times 0.08165 = 0.57155 \quad \text{(approx.)}
\]

Time taken:
\[
t = \frac{1}{7r} = \frac{1}{0.57155} \approx 1.75 \text{ hours}
\]

---

### **Answer:**

It would take approximately **1.75 hours (1 hour 45 minutes)** for seven people to complete the task, assuming all people work at roughly the same rate.

---

### **Summary:**

- The work rate per person is approximately \( r = \frac{1}{12} \) or 0.08 tasks per hour.
- For 7 people, the combined rate is \( 7r \).
- So, time required \( t = \frac{1}{7r} \approx 1.75 \) hours.
  
**Note:** Slight differences in rate calculations come from approximations in the problem; assuming identical rates yields a consistent answer near 1.75 hours.

# Congratulations!

That was a small, simple step in the direction of Agentic AI, with your new environment!

Next time things get more interesting...

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try this commercial application:<br/>
            First ask the LLM to pick a business area that might be worth exploring for an Agentic AI opportunity.<br/>
            Then ask the LLM to present a pain-point in that industry - something challenging that might be ripe for an Agentic solution.<br/>
            Finally have 3 third LLM call propose the Agentic AI solution. <br/>
            We will cover this at up-coming labs, so don't worry if you're unsure.. just give it a try!
            </span>
        </td>
    </tr>
</table>

In [13]:
# First create the messages:

messages = [{"role": "user", "content": "pick a business area that might be worth exploring for an Agentic AI opportunity."}]

# Then make the first call:
response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)


# Then read the business idea:

business_idea = response.choices[0].message.content
print(business_idea)

# And repeat! In the next message, include the business idea within the message

One promising business area to explore for an Agentic AI opportunity is *Personalized Healthcare and Wellness Management*.  
This area involves developing AI systems that can proactively manage an individual's health by continuously monitoring health data, providing tailored recommendations, scheduling preventive measures, and even coordinating with healthcare providers.  
  
Potential applications include:  
- Real-time health monitoring using wearable devices  
- Predictive analysis for early disease detection  
- Personalized lifestyle and nutrition guidance  
- Managing medication schedules and reminders  
- Proactively alerting users to potential health risks  
  
Exploring Agentic AI in this space could lead to more proactive, personalized, and efficient healthcare, ultimately improving health outcomes while reducing costs.


In [14]:
# First create the messages:

messages = [{"role": "user", "content": "present a pain-point in that industry - something challenging that might be ripe for an Agentic solution"}]

# Then make the first call:
response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)


# Then read the business idea:

pain_point = response.choices[0].message.content
print(pain_point)

# And repeat! In the next message, include the business idea within the message

Certainly! One significant pain point in the healthcare industry is the administrative burden on medical professionals. Physicians and healthcare staff often spend a substantial portion of their time on paperwork, billing, appointment scheduling, and documentation rather than on patient care. This inefficiency can lead to increased burnout, longer wait times for patients, and diminished quality of care.

An Agentic solution could leverage advanced AI-driven automation and intelligent agents to handle routine administrative tasks. For example, an autonomous assistant could pre-fill medical records, manage appointment scheduling based on clinician availability, triage and prioritize patient inquiries, and ensure accurate billing submissions. This would allow healthcare providers to focus more on patient interactions, improve operational efficiency, and ultimately enhance overall healthcare delivery.


In [15]:
# First create the messages:

messages = [{"role": "user", "content": "propose the Agentic AI solution"}]

# Then make the first call:
response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)


# Then read the business idea:

agentic_sol = response.choices[0].message.content
print(agentic_sol)

# And repeat! In the next message, include the business idea within the message

Certainly! Here is a proposed approach for implementing an Agentic AI solution:

**Agentic AI Solution Proposal**

**Overview:**
An Agentic AI is an autonomous system capable of making decisions, taking actions, and adapting to its environment to achieve specified goals. Such systems can be applied across various domains, including healthcare, finance, manufacturing, and customer service, to enhance efficiency, accuracy, and responsiveness.

**Core Components:**

1. **Autonomous Decision-Making Module:**
   - Leverages advanced machine learning models (e.g., reinforcement learning, deep learning) to evaluate scenarios and determine optimal actions.
   - Incorporates reasoning capabilities to handle complex, uncertain environments.

2. **Perception and Sensing Layer:**
   - Integrates sensors, data feeds, and natural language processing to perceive the environment or user inputs effectively.
   
3. **Knowledge Base and Context Management:**
   - Maintains domain knowledge, historical da